# Instalación de los paquetes - librerías necesarias

In [ ]:
!pip install flask==1.1.2
!pip install requests==2.25.1
!pip install flask-ngrok==0.0.25
#!pip install pyngrok==4.1.1
#!ngrok authtoken "2GjoLufuKVsVez81U4fOYA3zmTZ_7bi1T56kchTXC1vAvX7PL"

In [ ]:
import datetime
import json
import hashlib
import requests
from uuid import uuid4
from flask import Flask, jsonify, request
from urllib.parse import urlparse
from flask_ngrok import run_with_ngrok

# Creación de la criptomoneda

In [ ]:
class Blockchain:
  def __init__(self):
    self.chain = []
    self.transactions = []
    self.create_block(proof = 1, previous_hash = '0')
    self.nodes = set()

  def create_block(self, proof, previous_hash):
    """Creación de un nuevo bloque"""
    block = {
      'index': len(self.chain)+1,
      'timestamp': str(datetime.datetime.now()),
      'proof': proof,
      'previous_hash': previous_hash,
      'transaction': self.transactions
    }
    self.transactions = []
    self.chain.append(block)
    return block

  def get_previous_block(self):
    """Obtención del último bloque de la cadena"""
    return self.chain[-1]

  def proof_of_work(self, previous_proof):
    """Protocolo de consenso"""
    new_proof = 1
    check_proof = False
    while not check_proof:
      hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
      #En producción se usan operaciones más complejas, que sean no simétricas
      if hash_operation[:2] == '00':
        check_proof = True
      else: 
        new_proof +=1
    return new_proof

  def hash(self, block):
    """Calculo del hash de un bloque"""
    encoded_block = json.dumps(block, sort_keys = True).encode()
    hash_block = hashlib.sha256(encoded_block).hexdigest()    
    return hash_block
  
  def is_chain_valid(self, chain):
    """Determina si la cadena de bloques es valida"""
    previous_block = chain[0]
    block_index = 1
    while block_index < len(chain):
      block = chain[block_index]
      if block['previous_hash'] != self.hash(previous_block):
        return False
      previous_proof = previous_block['proof']
      proof = block['proof']
      hash_operation = hashlib.sha256(str(proof**2-previous_proof**2).encode()).hexdigest()
      if hash_operation[:2] != '00':
        return False
      previous_block = block
      block_index += 1
    return True

  
  def add_transaction(self, sender, receiver, amount):
    """ Realización de una transacción. """
    self.transactions.append({
        'sender': sender,
        'receiver': receiver,
        'amount': amount
      })
    previous_block = self.get_previous_block()
    return previous_block['index']+1

  def add_node(self, address):
    """Añadir un nuevo nodo en la blockchain"""
    parsed_url = urlparse(address)
    self.nodes.add(parsed_url.netloc)

  def replace_chain(self):
    """Reemplazo de la cadena por una cadena más larga, siempre y cuando sea válida"""
    network = self.nodes
    longest_chain = None
    max_length = len(self.chain)
    for node in network:
      response = requests.get(f'http://{node}/get_chain')
      if response.status_code == 200:
        length = response.json()['length']
        chain = response.json()['chain']
        if length > max_length and self.is_chain_valid(chain):
          max_length = length
          longest_chain = chain
    if longest_chain:
      self.chain = longest_chain
      return True
    return False


In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

#Crea la direccion del nodo en el puerto 5000
node_address = str(uuid4()).replace('-', '')

#Creamos la blockchain
blockchain = Blockchain()

@app.route('/mine_block', methods=['GET'])
def mine_block():
  """Minado de un bloque"""
  previous_block = blockchain.get_previous_block()
  previous_proof = previous_block['proof']
  proof = blockchain.proof_of_work(previous_proof)
  previous_hash = blockchain.hash(previous_block)
  blockchain.add_transaction(sender = node_address, receiver = 'Roberto Carlos', amount=10)
  block = blockchain.create_block(proof, previous_hash)
  response = {
      'message': 'Minaste un nuevo bloque',
      'index': block['index'],
      'timestamp': block['timestamp'],
      'proof': block['proof'],
      'previous_hash': block['previous_hash'],
      'transactions': block['transaction'] 
  }
  return jsonify(response), 200


@app.route('/get_chain', methods=['GET'])
def get_chain():
  response = {
      'chain': blockchain.chain,
      'length': len(blockchain.chain)
  }
  return jsonify(response), 200

@app.route('/is_valid', methods=['GET'])
def is_valid():
  """Comprobación si una cadena es valida"""
  is_valid = blockchain.is_chain_valid(blockchain.chain)
  if is_valid:
    response = {'message': 'La cadena es correcta'}
  else:
    response = {'message': 'La cadena es incorrecta'}
  return jsonify(response), 200

@app.route('/add_transaction', methods=['POST'])
def add_transaction():
  """ Añadir una nueva transaccion a la cadena de bloques """
  json = request.get_json()
  transaction_keys = ['sender', 'receiver', 'amount']
  if not all(key in json for key in transaction_keys):
    return 'Faltan algunos elementos de la transacción', 400
  index = blockchain.add_transaction(json['sender'], json['receiver'], json['amount'])
  response = {'message': f'La transacción será añadida al bloque {index}'}
  return jsonify(response), 200

#Descentralización de la cadena de bloques
#Conectar nuevos nodos
@app.route('/connect_node', methods=['POST'])
def connect_node():
  json = request.get_json()
  nodes = json.get('nodes')
  if nodes is None:
    return 'No hay nodos para añadir', 400
  for node in nodes:
    blockchain.add_node(node)
  response = {
      'message': 'Todos los nodos han sido conectados',
      'total_nodos': list(blockchain.nodes)
  }
  return jsonify(response), 201

@app.route('/replace_chain', methods=['GET'])
def replace_chain():
  """Reemplazar la cadena por la mas larga """
  is_chain_replaced = blockchain.replace_chain()
  if is_chain_replaced:
    response = {
        'message': 'Se remplazó por la cadena más larga',
        'new_chain': blockchain.chain
    }
  else:
    response = {
        'message': 'Todo correcto, tiene la blockchain correcta',
        'actual_chain': blockchain.chain
    }
  return jsonify(response), 200
  
@app.route('/', methods=['GET'])
def inicio():
  response = {
      'Message': 'Hola mundo'
  }
  return jsonify(response), 200

In [ ]:
app.run()